In [1]:
import tensorflow as tf
import torch; from torch import tensor
import numpy as np

2023-09-15 00:27:33.980383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
sigma:float = 0.1
min_sigma:float = 1e-3

A: tensor = tensor([[[0.2941, 0.8073, 0.5129, 0.7496, 0.3409, 0.6732],
         [0.9556, 0.2706, 0.5786, 0.9211, 0.4151, 0.5862],
         [0.3286, 0.5894, 0.1031, 0.2584, 0.5222, 0.4352]],

        [[0.0011, 0.8174, 0.6699, 0.6500, 0.9654, 0.7393],
         [0.4141, 0.9772, 0.8765, 0.0506, 0.5451, 0.1219],
         [0.9544, 0.2048, 0.0975, 0.1781, 0.4672, 0.7409]]]) # 2,3,6

B: tensor = tensor([[0.7245, 0.8128, 0.5149, 0.9719, 0.3836, 0.8181],
        [0.5655, 0.2984, 0.4779, 0.5923, 0.0746, 0.2779],
        [0.0882, 0.4241, 0.7142, 0.5485, 0.6926, 0.3199],
        [0.6485, 0.5530, 0.1981, 0.5301, 0.4493, 0.0664], 
        [0.6779, 0.8994, 0.1730, 0.4101, 0.5794, 0.1422]]) # 5,6

# Codigo Tensorflow

In [4]:
def kernel_tensor(A:tensor, B:tensor, sigma:float, min_sigma:float) -> tensor:
    '''
    Input:
        A: tensor of shape (bs, n, d)
        B: tensor of shape (m, d)
    Result:
        K: tensor of shape (bs, n, m)
    '''
    shape_A = tf.shape(A)
    shape_B = tf.shape(B)
    # norma, luego colocarla en forma de columna y elevarla al cuardadro
    A_norm = tf.norm(A, axis=-1)[..., tf.newaxis] ** 2 
    B_norm = tf.norm(B, axis=-1)[tf.newaxis, tf.newaxis, :] ** 2
    A_reshaped = tf.reshape(A, [-1, shape_A[2]])    
    AB = tf.matmul(A_reshaped, B, transpose_b=True) 
    AB = tf.reshape(AB, [shape_A[0], shape_A[1], shape_B[0]])
    dist2 = A_norm + B_norm - 2. * AB
    dist2 = tf.clip_by_value(dist2, 0., np.inf)
    sigma = tf.clip_by_value(sigma, min_sigma, np.inf) # TODO: Por qué realizar esto con tf
    K = tf.exp(-dist2 / (2. * sigma ** 2.)) # type: ignore
    return K

# From scrash in torch

In [5]:
def kernel_torch(A:tensor, B:tensor, sigma:float, min_sigma:float)-> tensor:

    res:tensor = A.unsqueeze(-2) - B
    norma:tensor = torch.sum(res,dim=-1 )
    valor_abs: tensor = torch.abs(norma)
    division: tensor = valor_abs/ (2. *sigma**2.)
    exponencial: tensor =  torch.exp(- division)
    return exponencial

### Prueba de los dos algoritmos

In [3]:
import random 
bs = random.randint(1, 10_00)
n = random.randint(1, 10_00)
d = random.randint(1, 10_00)
m = random.randint(1, 10_00)

In [257]:
for i in range(100):
    A = torch.rand(bs,n,d)
    B = torch.rand(m,d)
    tolerancia = 0.5

    kernel_1 = kernel_tensor(A,B, sigma, min_sigma)
    print(kernel_1)
    


tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

In [6]:
for i in range(100):
    A = torch.rand(bs,n,d)
    B = torch.rand(m,d)    
    kernel_2 = kernel_torch(A,B, sigma, min_sigma)
    print(kernel_2)

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 12790592100 bytes. Error code 12 (Cannot allocate memory)

In [ ]:

similares = torch.allclose(tensor(kernel_1.numpy()), kernel_2, atol=tolerancia)
print(similares)

### Primeras conclusiones

- Al parecer el algoritmo 2 utiliza mucho mas memoría que el algoritmo 1. Tanto que el computador no puede alojar tanta memoría.
- Por otro lado, al comparar cuando son demasiado grande, ya no hay una similaridad entre estos.